In [4]:
import sys, importlib
import os
import itertools
import json

import numpy as np
from scipy.spatial.distance import cosine
import statprof

sys.path.append("/cephfs/home/sorokin/DeepPavlov")
CONFIG_PATH = "config/DeepPavlov/morpho_ru_syntagrus_pymorphy.json"

from deeppavlov.core.commands.infer import build_model
from deeppavlov.models.embedders.fasttext_embedder import FasttextEmbedder
from ufal_udpipe import Model as udModel

In [7]:
from read_synsets import *
from read import *
from work import MixedUDPreprocessor, SyntaxTreeGraph
from save import analyze_scores, output_errors

2018-11-18 22:54:02.714 INFO in 'pymorphy2.opencorpora_dict.wrapper'['wrapper'] at line 16: Loading dictionaries from /cephfs/home/sorokin/env/lib/python3.6/site-packages/pymorphy2_dicts/data
2018-11-18 22:54:02.777 INFO in 'pymorphy2.opencorpora_dict.wrapper'['wrapper'] at line 20: format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168


In [8]:
# import sys, importlib
# importlib.reload(sys.modules["main_pairwise"])
from main_pairwise import PairwiseScorer

2018-11-18 22:54:13.938 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 175: [loading vocabulary from /cephfs/home/sorokin/DeepPavlov/download/morpho_tagger/UD2.0/ru_syntagrus/tag.dict]
2018-11-18 22:54:13.941 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 175: [loading vocabulary from /cephfs/home/sorokin/DeepPavlov/download/morpho_tagger/UD2.0/ru_syntagrus/char.dict]
2018-11-18 22:54:13.949 INFO in 'pymorphy2.opencorpora_dict.wrapper'['wrapper'] at line 16: Loading dictionaries from /cephfs/home/sorokin/env/lib/python3.6/site-packages/pymorphy2_dicts/data
2018-11-18 22:54:13.992 INFO in 'pymorphy2.opencorpora_dict.wrapper'['wrapper'] at line 20: format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168
2018-11-18 22:54:14.901 DEBUG in 'matplotlib'['__init__'] at line 410: $HOME=/home/sorokin
2018-11-18 22:54:14.902 DEBUG in 'matplotlib'['__init__'] at line 410: matplotlib data path /cephfs/home/sorokin/env/lib/python3.6/site-packages/matplotlib/mpl-data
2018-11-

Using TensorFlow backend.
2018-11-18 22:54:15.959 INFO in 'deeppavlov.models.morpho_tagger.network'['network'] at line 126: 99 symbols, 711 tags in CharacterTagger
2018-11-18 22:54:16.857 INFO in 'deeppavlov.models.morpho_tagger.network'['layer_utils'] at line 106: __________________________________________________________________________________________________
2018-11-18 22:54:16.858 INFO in 'deeppavlov.models.morpho_tagger.network'['layer_utils'] at line 104: Layer (type)                    Output Shape         Param #     Connected to                     
2018-11-18 22:54:16.858 INFO in 'deeppavlov.models.morpho_tagger.network'['layer_utils'] at line 108: ==================================================================================================
2018-11-18 22:54:16.859 INFO in 'deeppavlov.models.morpho_tagger.network'['layer_utils'] at line 104: input_1 (InputLayer)            (None, None, 32)     0                                            
2018-11-18 22:54:16.860 INFO in 

2018-11-18 22:54:16.879 INFO in 'deeppavlov.models.morpho_tagger.network'['layer_utils'] at line 104:                                                                  dense_2[0][0]                    
2018-11-18 22:54:16.880 INFO in 'deeppavlov.models.morpho_tagger.network'['layer_utils'] at line 170: __________________________________________________________________________________________________
2018-11-18 22:54:16.880 INFO in 'deeppavlov.models.morpho_tagger.network'['layer_utils'] at line 104: dropout_1 (Dropout)             (None, None, 1228)   0           concatenate_2[0][0]              
2018-11-18 22:54:16.881 INFO in 'deeppavlov.models.morpho_tagger.network'['layer_utils'] at line 170: __________________________________________________________________________________________________
2018-11-18 22:54:16.881 INFO in 'deeppavlov.models.morpho_tagger.network'['layer_utils'] at line 104: bidirectional_1 (Bidirectional) (None, None, 256)    1389568     dropout_1[0][0]              

In [9]:
OBLIGATORY_FIELDS = ["relations_file", "senses_file", "train_file", "test_file"]
DEFAULT_FIELDS = ["scorer", "embedders", "metrics", "use_derivates", "verbose",
                  "save_file", "save_scores_file", "load_file", "load_scores_file"]
DEFAULT_VALUES = [dict(), [], ["modified_lch"], False, 0] + [None] * 4

def read_config(infile):
    with open(infile, "r", encoding="utf8") as fin:
        data = json.load(fin)
    for key in OBLIGATORY_FIELDS:
        if key not in data:
            raise KeyError("Field {} must be present in config file".format(key))
    for key, value in zip(DEFAULT_FIELDS, DEFAULT_VALUES):
        if key not in data:
            data[key] = value
    return data

In [10]:
def read_embedders(data):
    embedders = []
    for infile, dim, model_type in data:
        if model_type == "fasttext":
            embedder = FasttextEmbedder(infile, dim=dim)
        else:
            raise NotImplementedError("{} embedder is not implemented".format(model_type))
        print("{} embedder loaded".format(model_type))
        embedders.append(embedder)
    return embedders

In [11]:
CONFIG_PATH = "config/pairwise/config_fasttext.json"
config = read_config(CONFIG_PATH)
synsets, synsets_encoding, graph, derivates = make_graph(config["relations_file"])

In [12]:
synsets_by_lemmas, lemmas_by_synsets = collect_synsets_for_lemmas(config["senses_file"])

In [13]:
ancestors_lists, D, depths = make_ancestors_lists(graph, derivates, use_derivates=config["use_derivates"])
embedders = read_embedders(config["embedders"])


10000 vertexes processed
20000 vertexes processed
30000 vertexes processed
40000 vertexes processed


2018-11-18 22:54:30.582 INFO in 'deeppavlov.models.embedders.fasttext_embedder'['fasttext_embedder'] at line 52: [loading fastText embeddings from `/cephfs/home/sorokin/data/embeddings/ft_native_300_ru_wiki_lenta_lower_case.bin`]


fasttext embedder loaded


In [14]:
scorer = PairwiseScorer(ancestors_lists, D, depths, synsets_by_lemmas,
                        synsets_encoding, config["metrics"], embedders=embedders,
                        verbose=config["verbose"], **config["scorer"])
scorer.load(config["load_file"], config["load_scores_file"])

In [15]:
sents, data, targets = read_data(config["train_file"], from_parses=True)

In [16]:
data_to_scorer = list(zip(data[::2], data[1::2]))
similarity_scores, word_scores =\
    scorer.score_sents(data_to_scorer, dump_file=config.get('train_distances_file'))

In [ ]:
class PairwiseTrainer:
    
    def __init__(use_pos=True, use_tag=True, use_syntax=True, syntax_depth=3):
        self.use_pos = use_pos or use_tag
        self.use_tag = use_tag
        self.use_syntax = use_syntax
        self.syntax_depth = syntax_depth
        
    def train(self, data, targets):
        possible_pos, possible_tags, possible_heads = set(), set(), set()
        for first, second, _ in data:
            for elem in first + second:
                possible_pos.add(elem[3])
                if elem[5] != "_":
                    feat_with_values = [x.split("=") for x in elem[5].split("|")]
                    for x in feat_with_values:
                        possible_tags.add("{}_{}_{}".format(pos, *x))
                possible_heads.add(elem[7])
        if self.use_pos:
            self.pos_labels_ = sorted(possible_pos)
            self.pos_codes_ = {label: code for code, label in enumerate(self.pos_labels_)}
            if self.use_tag:
                self.tags_ = sorted(possible_tags)
                self.tag_codes_ = {label: code for code, label in enumerate(self.tags_)}
        if self.use_syntax:
            self.heads_ = [None] + sorted(possible_heads)
            self.head_codes_ = {label: code for code, label in enumerate(self.heads_)}
        transformed_data = self.transform(data)
        self.build()
        self.train_model()
        return self
    
    def _transform_sent(self, data, indexes):
        answer, start = [[] for _ in data], 0
        if self.use_pos:
            for i, elem in enumerate(data):
                code = self.pos_codes_.get(elem[3])
                if code is not None:
                    answer[i].append(code)
            start += len(self.tag_codes_)
        if self.use_tags:
            for i, elem in enumerate(data):
                if elem[3] not in self.pos_codes_ or elem[5] == "_":
                    continue
                feat_with_values = [x.split("=") for x in elem[5].split("|")]
                feats = ["{}_{}_{}".format(elem[3], *x) for x in feat_with_values]
                for feat in feats:
                    code = self.tag_codes_.get(feat)
                    if code is not None:
                        answer[i].append(start+code)
                
                
    def transform(self, data):
        answer = []
        for first_data, second_data, first_indexes, second_indexes, distances in data:
            first_features = self._transform_sent(first_data, first_indexes)
            second_features = self._transform_sent(second_data, second_indexes)
            answer.append([distances] + first_data + second_data)
        return answer

In [ ]:
data[0][0]